In [1]:
from langchain_openai import ChatOpenAI
import os
from openai import api_key, base_url
from transformers import Qwen2_5_VLConfig

from langchain_core.output_parsers.string import StrOutputParser
from langchain.schema import output_parser
# llm = ChatOpenAI(model=)
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "sk-C3gsaWJaOsmjrNv1fIobc4zEEsEkD15T98j80Vc8mNuSE420")
# llm = ChatOpenAI(model="o3-mini", api_key=OPENAI_API_KEY, base_url="https://api.deepbricks.ai/v1/")
QWEN_API_KEY = os.getenv("QWEN_API_KEY", "sk-ypjnechwutigvbyglbkguukzsmzkkxfibauydwkbjrypwojd")
model = ChatOpenAI(model="Qwen/Qwen3-30B-A3B-Instruct-2507", api_key=QWEN_API_KEY, base_url="https://api.siliconflow.cn/v1")



output_parser = StrOutputParser()

/home/zpc/anaconda3/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.prompts import PromptTemplate


class Book(BaseModel):
    title: str = Field(description="书名")
    author: str = Field(description="作者")
    description: str = Field(description="简介")



In [8]:

query = "请给我介绍学习中国历史的经典书籍"
parser = JsonOutputParser(pydantic_object=Book)
format_instructions = parser.get_format_instructions()
format_instructions = """
输出应格式化为符合以下JSON的结构的JSON实例。
JSON结构'''
{
'title': '书名',
'author': '作者',
'description': '书的简介'
}'''
"""

prompt = PromptTemplate(
    template = "{format_instructions}\n{query}",
    input_variables = ["query"],
    partial_variables = {"format_instructions": format_instructions}
)

chain = prompt | model | parser


chain.invoke({"query": query})


{'title': '中国通史',
 'author': '吕思勉',
 'description': '《中国通史》是中国近代史学巨擘吕思勉的代表作之一，系统地梳理了从远古时代到近代中国的历史发展脉络。全书内容详实、结构严谨，注重历史因果与社会变迁的分析，是了解中国历史不可或缺的经典读物，适合对历史有深入兴趣的读者。'}

In [10]:
#自定义解析器
from langchain_core.messages.ai import AIMessage


def parser(ai_message: AIMessage) -> str:
    "Parse the AI message into a string"
    return ai_message.content.swapcase()

chain = model | parser

chain.invoke("hello")



'hELLO! hOW CAN i ASSIST YOU TODAY? 😊'